## Github link https://github.com/Christian-Vinicio/repo-seriestemporales-g4-pd.git

In [1]:


import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ==================================================================================
# 1. CARGA Y PREPARACIÓN DEL DATASET
# ==================================================================================

print("="*80)
print("CARGA DE DATOS")
print("="*80)

# Cargar el dataset 
df = pd.read_csv("../data/raw/ventas.csv")

# Convertir fecha a datetime
df['Fecha_Venta'] = pd.to_datetime(df['Fecha_Venta'], format='%d/%m/%Y')

print(f"Registros cargados: {len(df):,}")
print(f"Columnas: {list(df.columns)}")
print(f"\nPrimeras filas:")
print(df.head())

# ==================================================================================
# 2. CREACIÓN DE VARIABLES LAG Y CARACTERÍSTICAS TEMPORALES
# ==================================================================================

print("\n" + "="*80)
print("CREACIÓN DE LAGS Y CARACTERÍSTICAS TEMPORALES")
print("="*80)

# Ordenar por sucursal, producto y fecha
df = df.sort_values(['Codigo_Sucursal', 'Codigo_Producto', 'Fecha_Venta'])

# Crear características temporales
df['anio'] = df['Fecha_Venta'].dt.year
df['mes'] = df['Fecha_Venta'].dt.month
df['dia_semana'] = df['Fecha_Venta'].dt.dayofweek
df['dia_mes'] = df['Fecha_Venta'].dt.day
df['trimestre'] = df['Fecha_Venta'].dt.quarter

# Crear variable categórica de temporada
def asignar_temporada(mes):
    if mes in [12, 1, 2]:
        return 'Invierno'
    elif mes in [3, 4, 5]:
        return 'Primavera'
    elif mes in [6, 7, 8]:
        return 'Verano'
    else:
        return 'Otonio'

df['temporada'] = df['mes'].apply(asignar_temporada)

# Crear lags de ventas (1, 7, 14, 30 días)
# Los lags más útiles para forecasting son 1, 7, 14 y 30 días
print("\nCreando variables lag...")
for lag in [1, 7, 14, 30]:
    df[f'unidades_lag_{lag}'] = df.groupby(['Codigo_Sucursal', 'Codigo_Producto'])['Unidades_Vendidas'].shift(lag)
    df[f'total_lag_{lag}'] = df.groupby(['Codigo_Sucursal', 'Codigo_Producto'])['Total'].shift(lag)

# Rolling means (promedio móvil)
df['unidades_rolling_7'] = df.groupby(['Codigo_Sucursal', 'Codigo_Producto'])['Unidades_Vendidas'].transform(
    lambda x: x.rolling(window=7, min_periods=1).mean()
)
df['unidades_rolling_30'] = df.groupby(['Codigo_Sucursal', 'Codigo_Producto'])['Unidades_Vendidas'].transform(
    lambda x: x.rolling(window=30, min_periods=1).mean()
)

print(f"Total de columnas ahora: {len(df.columns)}")

# ==================================================================================
# 3. INGENIERÍA DE CARACTERÍSTICAS
# ==================================================================================

print("\n" + "="*80)
print("APLICACIÓN DE INGENIERÍA DE CARACTERÍSTICAS")
print("="*80)

np.random.seed(42)

# Eliminar valores aleatorios para demostrar imputación
indices_nulos_num = np.random.choice(df.index, size=int(0.02 * len(df)), replace=False)
df.loc[indices_nulos_num, 'unidades_lag_7'] = np.nan

indices_nulos_cat = np.random.choice(df.index, size=int(0.01 * len(df)), replace=False)
df.loc[indices_nulos_cat, 'temporada'] = np.nan

print(f"\nValores nulos introducidos para demostración:")
print(df.isnull().sum()[df.isnull().sum() > 0])

# ==================================================================================
# i. IMPUTACIÓN DE VARIABLES NUMÉRICAS
# ==================================================================================

print("\n" + "-"*80)
print("i. IMPUTACIÓN DE VARIABLES NUMÉRICAS")
print("-"*80)

# Criterio: Usar mediana para variables con lags debido a posibles outliers
columnas_numericas_imputar = ['unidades_lag_1', 'unidades_lag_7', 'unidades_lag_14', 
                               'unidades_lag_30', 'total_lag_1', 'total_lag_7', 
                               'total_lag_14', 'total_lag_30']

imputer_num = SimpleImputer(strategy='median')

for col in columnas_numericas_imputar:
    if col in df.columns and df[col].isnull().sum() > 0:
        df[col] = imputer_num.fit_transform(df[[col]])
        print(f"  - {col}: imputada con mediana")

print(f"\nValores nulos después de imputación numérica: {df[columnas_numericas_imputar].isnull().sum().sum()}")

# ==================================================================================
# ii. IMPUTACIÓN DE VARIABLES CATEGÓRICAS
# ==================================================================================

print("\n" + "-"*80)
print("ii. IMPUTACIÓN DE VARIABLES CATEGÓRICAS")
print("-"*80)

# Criterio: Usar moda para variables categóricas
columnas_categoricas_imputar = ['temporada']

imputer_cat = SimpleImputer(strategy='most_frequent')

for col in columnas_categoricas_imputar:
    if df[col].isnull().sum() > 0:
        df[col] = imputer_cat.fit_transform(df[[col]]).ravel()
        print(f"  - {col}: imputada con moda")

print(f"\nValores nulos después de imputación categórica: {df[columnas_categoricas_imputar].isnull().sum().sum()}")

# ==================================================================================
# iii. CODIFICACIÓN DE VARIABLES CATEGÓRICAS
# ==================================================================================

print("\n" + "-"*80)
print("iii. CODIFICACIÓN DE VARIABLES CATEGÓRICAS")
print("-"*80)

# Criterio: Label Encoding para variables ordinales, One-Hot para nominales

# Label Encoding para temporada (tiene cierto orden cíclico)
le_temporada = LabelEncoder()
df['temporada_encoded'] = le_temporada.fit_transform(df['temporada'])
print(f"  - temporada: Label Encoding aplicado")
print(f"    Mapeo: {dict(zip(le_temporada.classes_, le_temporada.transform(le_temporada.classes_)))}")

# One-Hot Encoding para día de la semana
df_dummies = pd.get_dummies(df['dia_semana'], prefix='dia_semana', drop_first=True)
df = pd.concat([df, df_dummies], axis=1)
print(f"  - dia_semana: One-Hot Encoding aplicado ({len(df_dummies.columns)} variables creadas)")

# Label Encoding para códigos de sucursal y producto
le_sucursal = LabelEncoder()
df['sucursal_encoded'] = le_sucursal.fit_transform(df['Codigo_Sucursal'])
print(f"  - Codigo_Sucursal: Label Encoding aplicado")

le_producto = LabelEncoder()
df['producto_encoded'] = le_producto.fit_transform(df['Codigo_Producto'])
print(f"  - Codigo_Producto: Label Encoding aplicado")

# ==================================================================================
# iv. TRATAMIENTO DE OUTLIERS
# ==================================================================================

print("\n" + "-"*80)
print("iv. TRATAMIENTO DE OUTLIERS")
print("-"*80)

# Criterio: Utilizar IQR para detectar y limitar outliers
def tratar_outliers_iqr(data, columna):
    Q1 = data[columna].quantile(0.25)
    Q3 = data[columna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    outliers_antes = ((data[columna] < limite_inferior) | (data[columna] > limite_superior)).sum()
    
    # Winsorización: limitar valores extremos
    data[f'{columna}_tratado'] = data[columna].clip(limite_inferior, limite_superior)
    
    return outliers_antes

columnas_outliers = ['Unidades_Vendidas', 'Total']

for col in columnas_outliers:
    n_outliers = tratar_outliers_iqr(df, col)
    print(f"  - {col}: {n_outliers} outliers detectados y tratados mediante winsorización")

# ==================================================================================
# v. TRANSFORMACIÓN DE VARIABLES NUMÉRICAS
# ==================================================================================

print("\n" + "-"*80)
print("v. TRANSFORMACIÓN DE VARIABLES NUMÉRICAS")
print("-"*80)

# Criterio: Aplicar log para reducir asimetría en variables de ventas
columnas_transformar = ['Unidades_Vendidas_tratado', 'Total_tratado']

for col in columnas_transformar:
    # Agregar constante pequeña para evitar log(0)
    df[f'{col}_log'] = np.log1p(df[col])
    
    # Calcular skewness antes y después
    skew_antes = df[col].skew()
    skew_despues = df[f'{col}_log'].skew()
    print(f"  - {col}:")
    print(f"    Skewness antes: {skew_antes:.3f}")
    print(f"    Skewness después de log: {skew_despues:.3f}")

# ==================================================================================
# vi. ESCALADO DE CARACTERÍSTICAS
# ==================================================================================

print("\n" + "-"*80)
print("vi. ESCALADO DE CARACTERÍSTICAS")
print("-"*80)

# Criterio: StandardScaler para variables con distribución normal
# MinMaxScaler para variables acotadas

# StandardScaler para lags y rolling means
columnas_standard = ['unidades_lag_1', 'unidades_lag_7', 'unidades_lag_14', 
                     'unidades_rolling_7', 'unidades_rolling_30']

scaler_standard = StandardScaler()
df_scaled_std = pd.DataFrame(
    scaler_standard.fit_transform(df[columnas_standard]),
    columns=[f'{col}_scaled' for col in columnas_standard],
    index=df.index
)
df = pd.concat([df, df_scaled_std], axis=1)
print(f"  - StandardScaler aplicado a {len(columnas_standard)} variables")

# MinMaxScaler para variables temporales
columnas_minmax = ['mes', 'dia_mes', 'dia_semana']

scaler_minmax = MinMaxScaler()
df_scaled_mm = pd.DataFrame(
    scaler_minmax.fit_transform(df[columnas_minmax]),
    columns=[f'{col}_normalized' for col in columnas_minmax],
    index=df.index
)
df = pd.concat([df, df_scaled_mm], axis=1)
print(f"  - MinMaxScaler aplicado a {len(columnas_minmax)} variables")

# ==================================================================================
# 7. RESUMEN DEL PROCESO Y MAPEO DE TRANSFORMACIONES
# ==================================================================================

print("\n" + "="*80)
print("RESUMEN DEL PIPELINE DE INGENIERÍA DE CARACTERÍSTICAS")
print("="*80)

pipeline_definido = """
PIPELINE DE TRANSFORMACIONES PARA FORECASTING DE VENTAS:

1. IMPUTACIÓN NUMÉRICA:
   Variables: unidades_lag_1, unidades_lag_7, unidades_lag_14, unidades_lag_30,
              total_lag_1, total_lag_7, total_lag_14, total_lag_30
   Método: Mediana
   Justificación: La mediana es robusta ante outliers que pueden existir en lags

2. IMPUTACIÓN CATEGÓRICA:
   Variables: temporada
   Método: Moda (valor más frecuente)
   Justificación: Preserva la distribución original de categorías

3. CODIFICACIÓN CATEGÓRICA:
   a) Label Encoding:
      - temporada → temporada_encoded
      - Codigo_Sucursal → sucursal_encoded
      - Codigo_Producto → producto_encoded
      Justificación: Variables que pueden actuar como exógenas en el modelo
   
   b) One-Hot Encoding:
      - dia_semana → dia_semana_1, dia_semana_2, ..., dia_semana_6
      Justificación: No existe relación ordinal entre días de la semana

4. TRATAMIENTO DE OUTLIERS:
   Variables: Unidades_Vendidas, Total
   Método: Winsorización con límites IQR (Q1-1.5*IQR, Q3+1.5*IQR)
   Resultado: Unidades_Vendidas_tratado, Total_tratado
   Justificación: Mantiene información sin distorsionar distribución

5. TRANSFORMACIÓN NUMÉRICA:
   Variables: Unidades_Vendidas_tratado, Total_tratado
   Método: Transformación logarítmica (log1p)
   Resultado: Unidades_Vendidas_tratado_log, Total_tratado_log
   Justificación: Reduce asimetría y estabiliza varianza

6. ESCALADO:
   a) StandardScaler (media=0, std=1):
      Variables: unidades_lag_1, unidades_lag_7, unidades_lag_14,
                unidades_rolling_7, unidades_rolling_30
      Justificación: Normaliza características para modelos sensibles a escala
   
   b) MinMaxScaler (rango [0,1]):
      Variables: mes, dia_mes, dia_semana
      Justificación: Variables temporales acotadas naturalmente

VARIABLES FINALES PARA MODELADO:
- Variables objetivo: Unidades_Vendidas, Total
- Variables exógenas: sucursal_encoded, producto_encoded
- Variables temporales: anio, mes, dia_semana, trimestre, temporada_encoded
- Variables lag: unidades_lag_1, unidades_lag_7, unidades_lag_14, unidades_lag_30
- Variables rolling: unidades_rolling_7, unidades_rolling_30
- Variables escaladas: sufijo _scaled y _normalized
- Variables transformadas: sufijo _log y _tratado

CONSIDERACIONES PARA FORECASTING:
- Los lags de 1, 7, 14 y 30 días capturan patrones diarios, semanales y mensuales
- Las variables de sucursal y producto permiten modelos jerárquicos
- El escalado facilita la convergencia en modelos de ML
- La transformación log estabiliza varianza en series temporales
"""

print(pipeline_definido)

# Guardar resumen estadístico
print("\n" + "="*80)
print("ESTADÍSTICAS FINALES DEL DATASET")
print("="*80)
print(f"\nDimensiones finales: {df.shape}")
print(f"Columnas creadas: {len(df.columns)}")
print(f"\nPrimeras columnas:")
print(df.columns.tolist()[:20])



# Mostrar muestra del dataset final
print("\nMuestra del dataset procesado:")
print(df[['Fecha_Venta', 'Codigo_Sucursal', 'Codigo_Producto', 
          'Unidades_Vendidas', 'unidades_lag_7', 'temporada_encoded',
          'Unidades_Vendidas_tratado_log']].head(10))

# uardar dataset procesado
df.to_csv('../data/processed/dataset_procesado.csv', index=False)
# print("\nDataset guardado como 'dataset_procesado.csv'")

CARGA DE DATOS
Registros cargados: 73,153
Columnas: ['Codigo_Sucursal', 'Fecha_Venta', 'Codigo_Producto', 'Unidades_Vendidas', 'Total']

Primeras filas:
   Codigo_Sucursal Fecha_Venta Codigo_Producto  Unidades_Vendidas   Total
0               11  2020-01-02         P105400                486  1324.5
1               12  2020-01-02         P407780                 80   218.0
2                1  2020-01-02         P407780                366  1000.5
3                1  2020-01-02         P185447                504   302.4
4                6  2020-01-02         P958288               1452  1102.8

CREACIÓN DE LAGS Y CARACTERÍSTICAS TEMPORALES

Creando variables lag...
Total de columnas ahora: 21

APLICACIÓN DE INGENIERÍA DE CARACTERÍSTICAS

Valores nulos introducidos para demostración:
temporada           731
unidades_lag_1      105
total_lag_1         105
unidades_lag_7     2184
total_lag_7         735
unidades_lag_14    1470
total_lag_14       1470
unidades_lag_30    3150
total_lag_30      